In [ ]:
library(Seurat)
library(Matrix)
library(tidyverse)

In [ ]:
source("../../util/sc_preprocess.R")

In [ ]:
# The following RDS files are products from import_DGE.R
dge_base   <- readRDS("vFeb202110x_original_rawdge.rds")
dge_todrop_NoAlloc <- readRDS("10x_to_drop_NoAlloc.drop.rds")

In [ ]:
bclist_todrop_NoAlloc <- read.table("barcode_correspondence_data//Fig4/10x_to_drop_NoAlloc_merge_correspondence.tsv",header=F)

In [ ]:
colnames(dge_base) <- gsub("-1","",colnames(dge_base))
dge_todrop_NoAlloc_new <- refresh_dge(dge_todrop_NoAlloc,bclist_todrop_NoAlloc$V1,bclist_todrop_NoAlloc$V2)


In [ ]:
dge_target_list <- list(dge_todrop_NoAlloc_new)
use_cells <- get_cells_used(dge_base,dge_target_list,percent_mito = 20)

dge_base_intersect   <- dge_base[,use_cells]
dge_todrop_NoAlloc_intersect  <- dge_todrop_NoAlloc_new[,use_cells]


In [ ]:
# intersection
ncol(dge_base_intersect)
ncol(dge_todrop_NoAlloc_intersect)

In [ ]:
orig.intersect.seu   <- CreateSeuratObject(dge_base_intersect)
todrop.NoAlloc.seu    <- CreateSeuratObject(dge_todrop_NoAlloc_intersect)


In [ ]:
high_var_genes <- getVarGenes_untilPCA(orig.intersect.seu,num_varFeatures = 5000)

In [ ]:
orig.intersect.seu   <- getVarGenes_untilPCA(orig.intersect.seu,var_genes=high_var_genes,num_varFeatures = 5000)
todrop.NoAlloc.seu    <- getVarGenes_untilPCA(todrop.NoAlloc.seu,var_genes=high_var_genes,num_varFeatures = 5000)


In [ ]:
orig.intersect.seu   <- seurat_processing_UMAP(orig.intersect.seu,dimlimit = 20,res = 0.6)
todrop.NoAlloc.seu  <- seurat_processing_UMAP(todrop.NoAlloc.seu,dimlimit = 20,res = 0.6)

In [ ]:
store_dir <- "seurat_obj/"

saveRDS(orig.intersect.seu,paste0(store_dir,"Mar2022_processed_10x_original_Seurat.intersectionWithNoAlloc.rds"))
saveRDS(todrop.NoAlloc.seu,paste0(store_dir,"Mar2022_processed_10x_to_drop_NoAlloc_Seurat.rds"))

In [ ]:
#Fig drawing
draw_umap(seu_base = orig.intersect.seu,
          seu_to10x = orig.intersect.seu,
          seu_todrop = todrop.NoAlloc.seu,
          col_seed = 6,
          w=6,
          h=6,
          is_pdf=F,
          outdir = "figure_out/",
          outname = "Mar2022_10x_NoAlloc")

In [ ]:
#Export CellCellDist
v.vln.todrop <- get_dist_pcaspace_scatter(orig.intersect.seu,
                                          todrop.NoAlloc.seu,
                                          outdir = "figure_out/",
                                          outname = "Mar2022_10x_to_drop_NoAlloc")